# Preparación de los nuevos datos para su posterior fusión

In [1]:
import pandas as pd

In [2]:
pip install pyarrow

**Vamos a cargar los dataset que hemos buscado en internet de una fuente fiable, se llaman train.parquet y test.parquet, nos interesaria ver el número de filas de cada archivo y también que nombre tienen las columnas y las etiquetas con las que se cataloga como spam o ham los mensajes.**

In [3]:
# --- 1. Cargar train.parquet ---
try:
    df_train_new = pd.read_parquet('train.parquet')
    print("--- df_train_new cargado ---")
    print(f"Filas: {len(df_train_new)}")
    print("Primeras 10 filas:") # Modificado para mostrar 10 filas
    print(df_train_new.head(10)) # Modificado para mostrar 10 filas
    print("\nColumnas y tipos de datos:")
    print(df_train_new.info())
    print(f"\nEtiquetas únicas en df_train_new['label'] (si existe): {df_train_new['label'].unique() if 'label' in df_train_new.columns else 'Columna label no encontrada'}")
    print(f"Etiquetas únicas en df_train_new['class_label'] (si existe): {df_train_new['class_label'].unique() if 'class_label' in df_train_new.columns else 'Columna class_label no encontrada'}")

except FileNotFoundError:
    print("ERROR: 'train.parquet' no encontrado. Asegúrate de que el archivo está en la misma carpeta que este notebook.")
    df_train_new = pd.DataFrame() # Crear DataFrame vacío para evitar errores posteriores
except Exception as e:
    print(f"ERROR al cargar 'train.parquet': {e}")
    df_train_new = pd.DataFrame()

print("\n" + "="*80 + "\n") # Separador visual

# --- 2. Cargar test.parquet ---
try:
    df_test_new = pd.read_parquet('test.parquet')
    print("--- df_test_new cargado ---")
    print(f"Filas: {len(df_test_new)}")
    print("Primeras 10 filas:") # Modificado para mostrar 10 filas
    print(df_test_new.head(10)) # Modificado para mostrar 10 filas
    print("\nColumnas y tipos de datos:")
    print(df_test_new.info())
    print(f"\nEtiquetas únicas en df_test_new['label'] (si existe): {df_test_new['label'].unique() if 'label' in df_test_new.columns else 'Columna label no encontrada'}")
    print(f"Etiquetas únicas en df_test_new['class_label'] (si existe): {df_test_new['class_label'].unique() if 'class_label' in df_test_new.columns else 'Columna class_label no encontrada'}")

except FileNotFoundError:
    print("ERROR: 'test.parquet' no encontrado. Asegúrate de que el archivo está en la misma carpeta que este notebook.")
    df_test_new = pd.DataFrame() # Crear DataFrame vacío
except Exception as e:
    print(f"ERROR al cargar 'test.parquet': {e}")
    df_test_new = pd.DataFrame()

print("\n--- Carga y Exploración Inicial Completada ---")
print("Ahora tienes 'df_train_new' y 'df_test_new' en memoria para inspeccionar.")

--- df_train_new cargado ---
Filas: 8175
Primeras 10 filas:
                                                text     label
0  hey I am looking for Xray baggage datasets can...  not_spam
1  "Get rich quick! Make millions in just days wi...      spam
2  URGENT MESSAGE: YOU WON'T BELIEVE WHAT WE HAVE...      spam
3  [Google AI Blog: Contributing Data to Deepfake...  not_spam
4  Trying to see if anyone already has timestamps...  not_spam
5  Bridging the gap between artificial intelligen...  not_spam
6  hi all any good leads on datasets for fuel pri...  not_spam
7   \n\nHi everyone,\n\nFor my thesis I'm looking...  not_spam
8  I'm looking for a large dataset of n-grams (pr...  not_spam
9  Check out these amazing weight loss pills! The...      spam

Columnas y tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8175 entries, 0 to 8174
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    8175 non-null   object
 1   

**Veamos como lucían nuestras columnas en nuestro dataset inicial spam.csv, porque necesitamos que luzcan igual que los archivos parquet para poder fusionarlos correctamente.**

In [4]:
df_original = pd.read_csv('spam.csv', sep=None, engine='python', encoding='latin-1')

# ¡Aquí está tu tabla! Sin nada más.
# Las columnas se llamarán según el contenido de la primera fila de tu CSV.
df_original.head(6)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN


**Vamos a ver cuantas filas tenía nuestro spam.csv:**

In [5]:
len(df_original)

5572

**Vamos a eliminar las columnas irrelevantes de spam.csv y a renombrar y reordenar las columnas v1 y v2 para conseguir el formato de los archivos parquet.**

In [6]:
df_original_raw = pd.read_csv('spam.csv', sep=None, engine='python', header=None, encoding='latin-1')

# --- Verificación y Reducción a 2 Columnas ---
# Forzamos que el DataFrame solo tenga las dos primeras columnas.
if df_original_raw.shape[1] > 2:
    print(f"Advertencia: 'spam.csv' detectado con {df_original_raw.shape[1]} columnas. Reduciendo a 2.")
    df_original_raw = df_original_raw.iloc[:, :2].copy() # Reasignamos el DataFrame a 2 columnas

# Asignamos los nombres de las columnas.
df_original_raw.columns = ['v1_label', 'v2_text']

# --- ¡NUEVO PASO CRUCIAL AQUÍ! ---
# Eliminamos la primera fila (índice 0) que contiene 'v1' y 'v2' como datos.
df_original_raw = df_original_raw.iloc[1:].copy()
print("Fila 'v1 v2' eliminada del DataFrame original.")


# --- CONTINUACIÓN DE LAS TRANSFORMACIONES SOLICITADAS ---

# 1. Eliminar las columnas Unnamed (este paso ahora es redundante).
# Seleccionamos solo las columnas que nos interesan: 'v1_label' y 'v2_text'.
df_transformado = df_original_raw[['v1_label', 'v2_text']].copy()

# 2. Reordenar y renombrar columnas
# Queremos 'v2_text' como primera columna y que se llame 'text'.
# Queremos 'v1_label' como segunda columna y que se llame 'label'.
df_transformado = df_transformado.rename(columns={'v2_text': 'text', 'v1_label': 'label'})
df_transformado = df_transformado[['text', 'label']] # Aseguramos el orden: 'text' primero, 'label' segundo

# 3. Sustituir valores en la columna 'label'
# Donde pone 'ham', debe poner 'not_spam'. 'spam' se queda igual.
df_transformado['label'] = df_transformado['label'].replace({'ham': 'not_spam'})

# 4. Mostrar las primeras 10 filas del DataFrame transformado en formato tabla
df_transformado.head(10)

Advertencia: 'spam.csv' detectado con 5 columnas. Reduciendo a 2.
Fila 'v1 v2' eliminada del DataFrame original.


,text,label
1,"Go until jurong point, crazy.. Available only ...",not_spam
2,Ok lar... Joking wif u oni...,not_spam
3,Free entry in 2 a wkly comp to win FA Cup fina...,spam
4,U dun say so early hor... U c already then say...,not_spam
5,"Nah I don't think he goes to usf, he lives aro...",not_spam
6,FreeMsg Hey there darling it's been 3 week's n...,spam
7,Even my brother is not like to speak with me. ...,not_spam
8,As per your request 'Melle Melle (Oru Minnamin...,not_spam
9,WINNER!! As a valued network customer you have...,spam
10,Had your mobile 11 months or more? U R entitle...,spam


**Comprobamos que no haya nulos en el nuevo dataframe**

In [8]:
# Suponiendo que df_transformado ya está cargado en tu entorno
# y tiene las columnas 'text' y 'label'.

# Muestra la cantidad de valores nulos por cada columna del DataFrame
print("Valores nulos en df_transformado por columna:")
print(df_transformado.isnull().sum())

Valores nulos en df_transformado por columna:
text     0
label    0
dtype: int64


**Compribamos que se conserve el número de filas que tenia el original**

In [9]:
len(df_transformado)

5572

**Guardo el nuevo df transformado en un csv llamado nuevo_spam.csv**

In [10]:
df_transformado.to_csv('nuevo_spam.csv', index=False, encoding='latin-1')

**Procedemos a combinar los 3 archivos para conseguir el resultado final, un dataset más rico en variedad de mensajes puesto que nuestro primer modelo cometía algunos errores clasificando spam ( de camino hemos eliminado filas duplicadas).**

In [19]:
# --- 1. Load nuevo_spam.csv ---
# Assuming nuevo_spam.csv is already correctly formatted with 'text' and 'label' columns
# and 'spam'/'not_spam' labels.
df_nuevo_spam = pd.read_csv('nuevo_spam.csv', encoding='latin-1')
print("--- 'nuevo_spam.csv' loaded ---")
print(f"Rows: {len(df_nuevo_spam)}")
print(f"Columns: {df_nuevo_spam.columns.tolist()}, Labels: {df_nuevo_spam['label'].unique()}")
# Comprobación de nulos en nuevo_spam.csv (debería ser 0)
print(f"Nulls in 'label' column of df_nuevo_spam: {df_nuevo_spam['label'].isnull().sum()}")


print("\n" + "="*80 + "\n") # Visual separator

# --- 2. Load train.parquet ---
df_train_new = pd.read_parquet('train.parquet')
print("--- 'train.parquet' loaded ---")
print(f"Rows: {len(df_train_new)}")
print(f"Columns: {df_train_new.columns.tolist()}, Labels: {df_train_new['label'].unique()}")
# Comprobación de nulos en train.parquet (debería ser 0)
print(f"Nulls in 'label' column of df_train_new: {df_train_new['label'].isnull().sum()}")


print("\n" + "="*80 + "\n") # Visual separator

# --- 3. Load test.parquet ---
df_test_new = pd.read_parquet('test.parquet')
print("--- 'test.parquet' loaded ---")
print(f"Rows: {len(df_test_new)}")
print(f"Columns: {df_test_new.columns.tolist()}, Labels: {df_test_new['label'].unique()}")
# Comprobación de nulos en test.parquet (debería ser 0)
print(f"Nulls in 'label' column of df_test_new: {df_test_new['label'].isnull().sum()}")


print("\n" + "="*80 + "\n") # Visual separator

# --- 4. Transform Parquet DataFrames to match nuevo_spam.csv format ---

# Transform df_train_new
df_train_transformed = df_train_new.copy()
# ¡CORREGIDO! ELIMINAMOS LA LÍNEA DE MAPEO, YA NO ES NECESARIA
# df_train_transformed['label'] = df_train_transformed['label'].map({0: 'not_spam', 1: 'spam'})
df_train_transformed = df_train_transformed[['text', 'label']] # Aseguramos el orden de las columnas

print("--- 'train.parquet' transformed (no mapping needed) ---")
print(f"Columns: {df_train_transformed.columns.tolist()}, Labels: {df_train_transformed['label'].unique()}")
print(f"Nulls in 'label' column of df_train_transformed: {df_train_transformed['label'].isnull().sum()}")


print("\n" + "="*80 + "\n") # Visual separator

# Transform df_test_new
df_test_transformed = df_test_new.copy()
# ¡CORREGIDO! ELIMINAMOS LA LÍNEA DE MAPEO, YA NO ES NECESARIA
# df_test_transformed['label'] = df_test_transformed['label'].map({0: 'not_spam', 1: 'spam'})
df_test_transformed = df_test_transformed[['text', 'label']] # Aseguramos el orden de las columnas

print("--- 'test.parquet' transformed (no mapping needed) ---")
print(f"Columns: {df_test_transformed.columns.tolist()}, Labels: {df_test_transformed['label'].unique()}")
print(f"Nulls in 'label' column of df_test_transformed: {df_test_transformed['label'].isnull().sum()}")


print("\n" + "="*80 + "\n") # Visual separator

# --- 5. Concatenate all three transformed DataFrames into a single new DataFrame ---
# We use ignore_index=True to reset the index of the combined DataFrame
df_final_combined = pd.concat([df_nuevo_spam, df_train_transformed, df_test_transformed], ignore_index=True)
print(f"--- All DataFrames concatenated (before dropping duplicates) ---")
print(f"Total rows: {len(df_final_combined)}")


print("\n" + "="*80 + "\n") # Visual separator

# --- 6. Remove duplicates from the final combined DataFrame ---
# Drop rows where both 'text' and 'label' are identical
df_final_combined.drop_duplicates(subset=['text', 'label'], inplace=True)
print(f"--- Final combined DataFrame (after dropping duplicates) ---")
print(f"Final rows: {len(df_final_combined)}")
print(f"Final labels: {df_final_combined['label'].unique()}")
# ¡CRUCIAL! Verificar nulos en el DataFrame final
print(f"Nulls in 'label' column of df_final_combined (final check): {df_final_combined['label'].isnull().sum()}")


print("\n" + "="*80 + "\n") # Visual separator

# --- 7. Display the first 10 rows of the final combined DataFrame ---
print("--- First 10 rows of the final combined DataFrame ---")
df_final_combined.head(10)

# --- 8. Save the final combined DataFrame to a new CSV file ---
df_final_combined.to_csv('spam_dataset.csv', index=False, encoding='utf-8')
print("\n--- Final combined DataFrame saved as 'spam_dataset.csv' ---")
print(f"File 'spam_dataset.csv' created with {len(df_final_combined)} rows.")

--- 'nuevo_spam.csv' loaded ---
Rows: 5572
Columns: ['text', 'label'], Labels: ['not_spam' 'spam']
Nulls in 'label' column of df_nuevo_spam: 0


--- 'train.parquet' loaded ---
Rows: 8175
Columns: ['text', 'label'], Labels: ['not_spam' 'spam']
Nulls in 'label' column of df_train_new: 0


--- 'test.parquet' loaded ---
Rows: 2725
Columns: ['text', 'label'], Labels: ['not_spam' 'spam']
Nulls in 'label' column of df_test_new: 0


--- 'train.parquet' transformed (no mapping needed) ---
Columns: ['text', 'label'], Labels: ['not_spam' 'spam']
Nulls in 'label' column of df_train_transformed: 0


--- 'test.parquet' transformed (no mapping needed) ---
Columns: ['text', 'label'], Labels: ['not_spam' 'spam']
Nulls in 'label' column of df_test_transformed: 0


--- All DataFrames concatenated (before dropping duplicates) ---
Total rows: 16472


--- Final combined DataFrame (after dropping duplicates) ---
Final rows: 15831
Final labels: ['not_spam' 'spam']
Nulls in 'label' column of df_final_combined 

In [20]:
print("Valores nulos en df_train_new por columna:")
print(df_train_new.isnull().sum())

Valores nulos en df_train_new por columna:
text     0
label    0
dtype: int64


In [21]:
print("Valores nulos en df_test_new por columna:")
print(df_test_new.isnull().sum())

Valores nulos en df_test_new por columna:
text     0
label    0
dtype: int64


In [22]:
print("Valores nulos en df_test_transformed por columna:")
print(df_test_transformed.isnull().sum())

Valores nulos en df_test_transformed por columna:
text     0
label    0
dtype: int64


In [23]:
print("Valores nulos en df_train_transformed por columna:")
print(df_train_transformed.isnull().sum())

Valores nulos en df_train_transformed por columna:
text     0
label    0
dtype: int64


**Comprobamos que la longitud de nuestro dataframe es correcta:**

In [26]:
len(df_final_combined)

15831

In [27]:
# Muestra la cantidad de valores nulos por cada columna del DataFrame df_final_combined
print("Valores nulos en df_final_combined por columna:")
print(df_final_combined.isnull().sum())

Valores nulos en df_final_combined por columna:
text     0
label    0
dtype: int64


**Guardamos en un csv nuestro nuevo dataset:**

In [28]:
# Este código lo guardará el nuevo df en un archivo CSV.
df_final_combined.to_csv('spam_dataset.csv', index=False, encoding='utf-8')

# Mensaje de confirmación (esto se imprimirá en la salida de la celda)
print("DataFrame 'df_final_combined' guardado como 'spam_dataset.csv'.")

DataFrame 'df_final_combined' guardado como 'spam_dataset.csv'.


**María del Carmen Martín Rodríguez, 16/07/2025**